In [1]:
from typing import Dict, List, Iterator

In [2]:
import numpy as np
import random
from pathlib import Path
from PIL import Image

In [3]:
def load_and_process_image(img_path: Path, rot: float):
    with Image.open(img_path) as img:
        img = img.rotate(rot)
        img = img.resize((28, 28))
        
        data = np.asarray(img, dtype=np.float32)
        data = np.transpose(data)  # (width, height) => (height, width)
        data = data.reshape((28, 28, 1))  # (height, width) => (height, width, channel)
        return data

result = load_and_process_image(Path("data/omniglot/data/Angelic/character15/0979_01.png"), 90)
assert result.shape == (28, 28, 1)
assert result.dtype == np.float32

In [4]:
OMNIGLOT_CACHE = {}


def load_class_image(data_dir: Path, clazz: str) -> List[np.ndarray]:
    if clazz not in OMNIGLOT_CACHE:
        alphabet, character, raw_rot = clazz.split('/')
        rot = float(raw_rot[3:])

        image_dir = data_dir / 'data' / alphabet / character

        class_images = sorted(image_dir.glob('*.png'))

        if len(class_images) == 0:
            raise Exception("No images found for omniglot class {} at {}. Did you run download_omniglot.sh first?".format(clazz, data_dir))

        image_list = [load_and_process_image(img_path, rot) for img_path in class_images]
            
        OMNIGLOT_CACHE[clazz] = image_list

    return OMNIGLOT_CACHE[clazz]

result = load_class_image(Path("data/omniglot"), "Angelic/character01/rot000")
assert len(result) == 20

In [5]:
def read_images(data_dir: Path, split: str) -> Dict[str, List[np.ndarray]]:
    split_dir = data_dir / "splits" / "vinyals"
    
    class_names = []
    with open(split_dir / "{:s}.txt".format(split), 'r') as f:
        for class_name in f.readlines():
            class_names.append(class_name.rstrip('\n'))
            
    images = {clazz: load_class_image(data_dir, clazz) for clazz in class_names}
    
    return images


result = read_images(Path("data/omniglot"), "train")
assert len(result) == 4112

In [6]:
def extract_episode(data_dir: Path, split: str, n_support, n_query) -> Dict[str, Dict[str, List[np.ndarray]]]:
    data = read_images(data_dir, split)

    result = {}
    for clazz, images in data.items():
        random.shuffle(images)
        xs = images[:n_support]
        xq = images[n_support:n_support + n_query]
        
        result[clazz] = {"xs": np.stack(xs), "xq": np.stack(xq)}
        
    return result

result = extract_episode(Path("data/omniglot"), "train", 5, 3)
assert len(result) == 4112

some_key = list(result.keys())[0]
assert result[some_key]["xs"].shape == (5, 28, 28, 1)
assert result[some_key]["xq"].shape == (3, 28, 28, 1)

In [7]:
class EpisodeBatcher:
    def __init__(self, data_dir: Path, split: str, n_support, n_query, n_episodes, n_way):
        self.episode_config = {"data_dir": data_dir, "split": split, "n_support": n_support, "n_query": n_query}
        self.n_episodes = n_episodes
        self.n_way = n_way
    
    def __len__(self):
        return self.n_episodes
    
    def __iter__(self):
        episode = extract_episode(**self.episode_config)
        class_list = list(episode.keys())
        for e in range(self.n_episodes):
            random.shuffle(class_list)
            n_way_class = class_list[:self.n_way]
            data = [v for k, v in episode.items() if k in n_way_class]
            
            xs = np.stack([i["xs"] for i in data])
            xq = np.stack([i["xq"] for i in data])
            
            yield xs, xq

result = list(EpisodeBatcher(Path("data/omniglot"), "train", 5, 5, 6, 5))
assert len(result) == 6

for xs, xq in EpisodeBatcher(Path("data/omniglot"), "train", 5, 3, 6, 10):    
    assert xs.shape == (10, 5, 28, 28, 1)
    assert xq.shape == (10, 3, 28, 28, 1)
    
    break

In [8]:
def pairwise_dist(A, B):  
    """
    Computes pairwise distances between each elements of A and each elements of B.
    Args:
    A,    [m,d] matrix
    B,    [n,d] matrix
    Returns:
    D,    [m,n] matrix of pairwise distances
    """
    # squared norms of each row in A and B
    na = tf.reduce_sum(tf.square(A), 1)
    nb = tf.reduce_sum(tf.square(B), 1)

    # na as a row and nb as a co"lumn vectors
    na = tf.reshape(na, [-1, 1])
    nb = tf.reshape(nb, [1, -1])

    # return pairwise euclidead difference matrix
    D = tf.sqrt(tf.maximum(na - 2*tf.matmul(A, B, False, True) + nb, 0.0))
    return D

In [9]:
import tensorflow as tf


class Img2Vec(tf.keras.Model):
    """
    Img2Vec CNN which takes image of dimension (28x28x?) and return column vector length 64
    """

    def __init__(self):
        super(Img2Vec, self).__init__()

        self.convnet1 = self.sub_block()
        self.convnet2 = self.sub_block()
        self.convnet3 = self.sub_block()
        self.convnet4 = self.sub_block()

    def sub_block(self, out_channels=64, kernel_size=3):
        block = tf.keras.models.Sequential(
            [
                tf.keras.layers.Conv2D(out_channels, kernel_size, padding="same"),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.ReLU(),
                tf.keras.layers.MaxPool2D(),
            ]
        )
        return block

    def call(self, x):
        x = self.convnet1(x)
        x = self.convnet2(x)
        x = self.convnet3(x)
        x = self.convnet4(x)
        x = tf.keras.layers.Flatten()(x)
        return x

In [25]:
img2vec = Img2Vec()
optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_acc = tf.keras.metrics.Mean(name='train_acc')

def train_step(xs, xq):
    n_class = xs.shape[0]
    assert xq.shape[0] == n_class
    n_support = xs.shape[1]
    n_query = xq.shape[1]
    
    with tf.GradientTape() as tape:
        target_idx = tf.broadcast_to(
            tf.reshape(tf.range(0, n_class), (-1, 1, 1)),
            (n_class, n_query, 1)
        )

        x = np.concatenate([
            np.reshape(xs, [n_class * n_support, *xs.shape[2:]]),
            np.reshape(xq, [n_class * n_query, *xq.shape[2:]])
        ])

        z = img2vec(x)

        z_proto = tf.math.reduce_mean(tf.reshape(z[:n_class * n_support], (n_class, n_support, -1)), 1)

        z_q = z[n_class * n_support:]

        dists = pairwise_dist(z_q, z_proto)

        log_p_y = tf.reshape(tf.nn.log_softmax(-1 * dists, 1), (n_class, n_query, -1))

        loss_val = -1 * tf.reduce_mean(tf.reshape(tf.gather(log_p_y, target_idx, axis=2, batch_dims=2), (-1, )))
        
        y_hat = tf.math.argmax(log_p_y, 2, output_type=tf.int32)
        acc_val = tf.reduce_mean(tf.cast(tf.math.equal(y_hat, tf.squeeze(target_idx)), tf.float32))

    
    gradients = tape.gradient(loss_val, img2vec.trainable_variables)
    optimizer.apply_gradients(zip(gradients, img2vec.trainable_variables))

    train_loss(loss_val)
    train_acc(acc_val)

In [ ]:
import time
from tqdm import tqdm

EPOCHES = 10000
EPISODES = 100

for epoch in range(EPOCHES):
    train_loss.reset_states()
    train_acc.reset_states()
        
    episode_generator = EpisodeBatcher(Path("data/omniglot"), "train", 5, 5, EPISODES, 60)
    for episode in tqdm(episode_generator, desc="Batch {}".format(epoch)):
        train_step(*episode)
        
    template = 'Epoch {}, Loss: {}, Acc: {}'
    print(template.format(epoch + 1, train_loss.result(), train_acc.result()))

Batch 1:   0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Epoch 1, Loss: 1.6581666469573975, Acc: 0.587766706943512


Batch 2:   0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Epoch 2, Loss: 0.6782087087631226, Acc: 0.8044664859771729


Batch 3:   0%|                                                                                          | 0/100 [00:00<?, ?it/s]

...

Batch 46:   0%|                                                                                         | 0/100 [00:00<?, ?it/s]

Epoch 46, Loss: 0.06509291380643845, Acc: 0.9794996380805969


Batch 47:   0%|                                                                                         | 0/100 [00:00<?, ?it/s]

Epoch 47, Loss: 0.061215486377477646, Acc: 0.9808666110038757


Batch 47:   8%|██████▏                                                                       | 8/100 [35:14<2:44:45, 107.45s/it]